In [1]:
import pandas as pd
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  

import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

import numpy as np
import matplotlib.pyplot as plt
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/nlp project/data.csv')
data

Mounted at /content/drive


,Unnamed: 0,targetCategory,post,y
0,0,race,RT @iBeZo: Stupid fucking nigger LeBron. You f...,1
1,1,race,I need a girl from Jamaica I can't fuck with t...,1
2,2,gender,Molly make bitches crazy,2
3,3,race,This niggga said I be branding bitches &#12856...,1
4,4,race,It's funny when u get a black female mad they ...,1
...,...,...,...,...
12075,12075,gender,Does anyone else have the suspicion that moder...,2
12076,12076,disabled,We need World War III\nThen all Chads and Norm...,5
12077,12077,gender,Roasties can look like this and STILL be consi...,2
12078,12078,social,Help me find a video\nI'm looking for a video ...,6


In [3]:
data[data['targetCategory']=="social"]['post'].iloc[7]

'Ok this is funny... A proud liberal who wants to end racism calls Tim Scott an Uncle Tom Lolz lib idiots #tcot http://t.co/y6X6eagw20'

In [4]:
X = data.iloc[:, [2,1]]
X

,post,targetCategory
0,RT @iBeZo: Stupid fucking nigger LeBron. You f...,race
1,I need a girl from Jamaica I can't fuck with t...,race
2,Molly make bitches crazy,gender
3,This niggga said I be branding bitches &#12856...,race
4,It's funny when u get a black female mad they ...,race
...,...,...
12075,Does anyone else have the suspicion that moder...,gender
12076,We need World War III\nThen all Chads and Norm...,disabled
12077,Roasties can look like this and STILL be consi...,gender
12078,Help me find a video\nI'm looking for a video ...,social


In [5]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()



def cleantext(text):
    text= text.lower()
    text= re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text= re.sub(r"http\S+", "",text)
    text= re.sub(r"http", "",text)
    
    punctuations= '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text= text.replace(p, '')
        
    text= [word.lower() for word in text.split() if word.lower() not in sw]
    
    text= [lemmatizer.lemmatize(word) for word in text]
    
    text = " ".join(text)
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    text= emoji_pattern.sub(r'',text)
    
    return text

In [6]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:
X['post'] = X['post'].apply(lambda x: cleantext(x))
 
X.head() 

<ipython-input-7-b1fd811f9254>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['post'] = X['post'].apply(lambda x: cleantext(x))


,post,targetCategory
0,rt ibezo stupid fucking nigger lebron flopping...,race
1,need girl jamaica fuck basic black bitch,race
2,molly make bitch crazy,gender
3,niggga said branding bitch,race
4,funny u get black female mad start acting ghet...,race


In [8]:
from sklearn.preprocessing import LabelEncoder# Initialize the Label Encoder.
le = LabelEncoder()

# Encode the categories
X['TargetCategory_enc'] = le.fit_transform(data['targetCategory'])

# Display the first five rows again to see the result
display(X.head())

# Print the datatypes
print(X.dtypes)

<ipython-input-8-6a7a233d6453>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['TargetCategory_enc'] = le.fit_transform(data['targetCategory'])


,post,targetCategory,TargetCategory_enc
0,rt ibezo stupid fucking nigger lebron flopping...,race,4
1,need girl jamaica fuck basic black bitch,race,4
2,molly make bitch crazy,gender,3
3,niggga said branding bitch,race,4
4,funny u get black female mad start acting ghet...,race,4


post                  object
targetCategory        object
TargetCategory_enc     int64
dtype: object


In [9]:
from sklearn.model_selection import train_test_split

A = X.post
y = X.targetCategory

X_train, X_test, y_train, y_test = train_test_split(A, y, test_size=0.2, random_state = 42)

#TFIDF

In [46]:
vectorizer = TfidfVectorizer()
model = vectorizer.fit_transform(X_train)
vectorizer.get_feature_names_out()
print(model.shape)

(9664, 12914)


## tfidf + logistic regression

In [61]:
m = make_pipeline(TfidfVectorizer(), LogisticRegression(multi_class = 'multinomial',penalty='l2'))
m.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(multi_class='multinomial'))])

In [62]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['social', 'victim', 'disabled', ..., 'gender', 'culture', 'race'],
      dtype=object)

In [63]:
confusionmat_embeded = confusion_matrix(y_test, y_pred_em)
confusionmat_embeded

array([[ 10,   4,   1,  29,   5,   0,  30],
       [  0, 306,   0,  31,  54,   1,  31],
       [  0,   4,  91,  29,  14,   0,  21],
       [  0,   3,   0, 551,  31,   2,  35],
       [  1,  40,   2,  28, 614,   2,  17],
       [  1,  11,   1,  38,  21,  38,   7],
       [  1,  34,   2,  61,  33,   1, 180]])

In [64]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 1 accuracy:", accuracy)
print("Architecture 1 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 1 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 1 accuracy: 0.7408940397350994
Architecture 1 microaveraged F1 score: 0.7408940397350994
Architecture 1 macroaveraged F1 score: 0.6194674149163898


## tfidf + naive bayesian

In [51]:
m = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [52]:
m.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [53]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['race', 'gender', 'race', ..., 'gender', 'culture', 'race'],
      dtype='<U8')

In [54]:
confusionmat_embeded = confusion_matrix(y_test, y_pred_em)
confusionmat_embeded

array([[  0,   3,   0,  43,  27,   0,   6],
       [  0, 227,   0,  44, 146,   0,   6],
       [  0,   7,  12,  57,  82,   0,   1],
       [  0,   1,   0, 557,  62,   0,   2],
       [  0,  18,   0,  51, 631,   0,   4],
       [  0,  10,   0,  53,  52,   1,   1],
       [  0,  28,   0, 113, 113,   0,  58]])

In [55]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 1 accuracy:", accuracy)
print("Architecture 1 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 1 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 1 accuracy: 0.6150662251655629
Architecture 1 microaveraged F1 score: 0.6150662251655629
Architecture 1 macroaveraged F1 score: 0.3579796960450362


# TF-IDF + adaboost

In [65]:
from sklearn.ensemble import AdaBoostClassifier

In [66]:
m = make_pipeline(TfidfVectorizer(), AdaBoostClassifier())
m.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('adaboostclassifier', AdaBoostClassifier())])

In [67]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['social', 'victim', 'disabled', ..., 'gender', 'victim', 'race'],
      dtype=object)

In [68]:
confusionmat_embeded = confusion_matrix(y_test, y_pred_em)
confusionmat_embeded

array([[ 18,   0,   1,  12,   2,   1,  45],
       [  0, 210,   5,  16,  13,   6, 173],
       [  1,   0,  87,   5,   5,   1,  60],
       [  3,   1,   3, 424,  35,  10, 146],
       [  2,  11,   6,  23, 485,  13, 164],
       [  0,   2,   1,   7,   7,  54,  46],
       [  3,  17,   6,  30,   6,   2, 248]])

In [69]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 1 accuracy:", accuracy)
print("Architecture 1 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 1 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 1 accuracy: 0.6316225165562914
Architecture 1 microaveraged F1 score: 0.6316225165562914
Architecture 1 macroaveraged F1 score: 0.5832028533723388


# TF-IDF + MLP

In [70]:
m = make_pipeline(TfidfVectorizer(), MLPClassifier(hidden_layer_sizes=(200,100,20,),verbose=True))
m.fit(X_train, y_train)

Iteration 1, loss = 1.78688037
Iteration 2, loss = 1.15084369
Iteration 3, loss = 0.59197283
Iteration 4, loss = 0.30520451
Iteration 5, loss = 0.15765418
Iteration 6, loss = 0.09207642
Iteration 7, loss = 0.05698319
Iteration 8, loss = 0.04162951
Iteration 9, loss = 0.03203687
Iteration 10, loss = 0.02704706
Iteration 11, loss = 0.02284444
Iteration 12, loss = 0.01969464
Iteration 13, loss = 0.01895375
Iteration 14, loss = 0.01806552
Iteration 15, loss = 0.01594005
Iteration 16, loss = 0.01414672
Iteration 17, loss = 0.01419048
Iteration 18, loss = 0.01432214
Iteration 19, loss = 0.01478314
Iteration 20, loss = 0.01363297
Iteration 21, loss = 0.01279530
Iteration 22, loss = 0.01258469
Iteration 23, loss = 0.01140829
Iteration 24, loss = 0.01390229
Iteration 25, loss = 0.01250541
Iteration 26, loss = 0.01056900
Iteration 27, loss = 0.01226959
Iteration 28, loss = 0.01227761
Iteration 29, loss = 0.01173193
Iteration 30, loss = 0.01183469
Iteration 31, loss = 0.01127859
Iteration 32, los

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=(200, 100, 20),
                               verbose=True))])

In [71]:
y_pred_em = m.predict(X_test)
y_pred_em

array(['social', 'victim', 'disabled', ..., 'gender', 'culture', 'race'],
      dtype='<U8')

In [72]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 2 accuracy:", accuracy)
print("Architecture 2 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 2 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 2 accuracy: 0.6639072847682119
Architecture 2 microaveraged F1 score: 0.6639072847682119
Architecture 2 macroaveraged F1 score: 0.5763868672066618


## Glove + RNN

In [10]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [11]:
A = X.post
y = X.TargetCategory_enc

X_train, X_test, y_train, y_test = train_test_split(A, y, test_size=0.2, random_state = 42)

In [12]:
sw_set=stopwords.words('english')+list(string.punctuation)
glove_file = datapath('/content/drive/MyDrive/nlp project/glove.6B.100d.txt')
word2vec_text_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_text_file)
model_em = KeyedVectors.load_word2vec_format(word2vec_text_file)

In [13]:
## use my impelementation of calc_doc_embedding
OOV_embed = dict()
def calc_doc_embedding(doc, model, embed_size, OOV_embed):
    
    words = doc.split(" ")
    doc_embed = np.zeros((100,100))
    for i in range(len(words)):
        word = words[i]
        if word not in stopset:
            try: 
                embed = model.get_vector(word)
            except:
                try:
                    embed = OOV_embed[word]
                except:
                    embed = np.random.rand(embed_size)
                    OOV_embed[word] = embed
                    
            doc_embed[i] = embed

    return doc_embed

stopset = stopwords.words('english') + list(string.punctuation)

In [14]:
def calc_doc_length(doc, model, embed_size, OOV_embed):
    
    words = doc.split(" ")
    doc_len = 0
    for i in range(len(words)):
        word = words[i]
        if word not in stopset:
            try: 
                embed = model.get_vector(word)
            except:
                try:
                    embed = OOV_embed[word]
                except:
                    embed = np.random.rand(embed_size)
                    OOV_embed[word] = embed
                    
            doc_len += 1

    return doc_len

In [15]:
X_train_len = np.asarray([calc_doc_length(d, model_em, 100, OOV_embed) for d in X_train])
X_test_len = np.asarray([calc_doc_length(d, model_em, 100, OOV_embed) for d in X_test])
X_train_embeded = np.array([calc_doc_embedding(d, model_em, 100, OOV_embed) for d in X_train])
X_test_embeded = np.array([calc_doc_embedding(d, model_em, 100, OOV_embed) for d in X_test])

In [16]:
class CustomDataset(Dataset):
    def __init__(self, X, Y, x_length):
        self.X = X
        self.y = Y
        self.x_length = x_length
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return  torch.from_numpy(self.X[idx].astype(np.float64)), torch.from_numpy(self.y[idx].astype(np.int32)),self.x_length[idx]

In [17]:
def to_one_hot(labels, dimension=7):
  results = np.zeros(dimension)
  results[labels-1] = 1
  return results.astype(int)

y_train_embeded = y_train.apply(to_one_hot).values
y_test_embeded = y_test.apply(to_one_hot).values

In [18]:
training_set = CustomDataset(X_train_embeded, y_train_embeded, X_train_len)
testing_set = CustomDataset(X_test_embeded, y_test_embeded, X_test_len)
batch_size = 256
training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_set, batch_size=batch_size)

In [24]:
class RNN_variable_input(torch.nn.Module) :
    def __init__(self, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 7)
        
    def forward(self, x, x_length):
        x_pack = pack_padded_sequence(x, x_length, batch_first=True, enforce_sorted=False)
        out_pack, ht = self.rnn(x_pack)
        out = self.linear(ht[-1])
        return out

In [25]:
model = RNN_variable_input(100, 100)
model.to(device)

RNN_variable_input(
  (rnn): RNN(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=7, bias=True)
)

In [26]:
def BCE_loss(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=0.0001)
loss_fn = BCE_loss
loss_fu = torch.nn.CrossEntropyLoss()

In [27]:
def train(epoch):
    model.train()
    for X,y,l in training_loader:

        outputs = model(X.float().to(device),l)

        optimizer.zero_grad()

        loss = loss_fn(outputs.to(device), y.float().to(device))

        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [28]:
EPOCHS = 100
for epoch in range(EPOCHS):
  print(epoch)
  train(epoch)

0
Epoch: 0, Loss:  0.6809655427932739
Epoch: 0, Loss:  0.6789328455924988
Epoch: 0, Loss:  0.6738556623458862
Epoch: 0, Loss:  0.6704870462417603
Epoch: 0, Loss:  0.6714257597923279
Epoch: 0, Loss:  0.6680448651313782
Epoch: 0, Loss:  0.6643833518028259
Epoch: 0, Loss:  0.661514163017273
Epoch: 0, Loss:  0.6595999598503113
Epoch: 0, Loss:  0.6591062545776367
Epoch: 0, Loss:  0.6554233431816101
Epoch: 0, Loss:  0.6517355442047119
Epoch: 0, Loss:  0.6497365832328796
Epoch: 0, Loss:  0.6470584869384766
Epoch: 0, Loss:  0.6428293585777283
Epoch: 0, Loss:  0.6413320899009705
Epoch: 0, Loss:  0.64324951171875
Epoch: 0, Loss:  0.6370757222175598
Epoch: 0, Loss:  0.6361241340637207
Epoch: 0, Loss:  0.6328757405281067
Epoch: 0, Loss:  0.6310551166534424
Epoch: 0, Loss:  0.6261991858482361
Epoch: 0, Loss:  0.6235845685005188
Epoch: 0, Loss:  0.6217937469482422
Epoch: 0, Loss:  0.6212717890739441
Epoch: 0, Loss:  0.6148406863212585
Epoch: 0, Loss:  0.612237811088562
Epoch: 0, Loss:  0.60826110839

In [29]:
def validation():
    model.eval()
    with torch.no_grad():
        for X,y,l in test_loader:
            outputs = model(X.float().to(device),l)
    return outputs, y
outputs, y_test = validation()
def output2predict(outputs, dim = 7):
  c = np.argmax(outputs)
  p = np.zeros(dim).astype(int)
  p[c] = 1
  return p
y_pred_em = np.array([output2predict(output.cpu()) for output in outputs])
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 2 accuracy:", accuracy)
print("Architecture 2 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 2 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 2 accuracy: 0.6696428571428571
Architecture 2 microaveraged F1 score: 0.6696428571428571
Architecture 2 macroaveraged F1 score: 0.40349579682238934


## golve + lstm

In [30]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [31]:
A = X.post
y = X.TargetCategory_enc

X_train, X_test, y_train, y_test = train_test_split(A, y, test_size=0.2, random_state = 42)

In [32]:
sw_set=stopwords.words('english')+list(string.punctuation)

In [33]:
glove_file = datapath('/content/drive/MyDrive/nlp project/glove.6B.100d.txt')
word2vec_text_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_text_file)

(400001, 100)

In [34]:
model_em = KeyedVectors.load_word2vec_format(word2vec_text_file)

In [35]:
## use my impelementation of calc_doc_embedding
OOV_embed = dict()
def calc_doc_embedding(doc, model, embed_size, OOV_embed):
    
    words = doc.split(" ")
    doc_embed = np.zeros((100,100))
    for i in range(len(words)):
        word = words[i]
        if word not in stopset:
            try: 
                embed = model.get_vector(word)
            except:
                try:
                    embed = OOV_embed[word]
                except:
                    embed = np.random.rand(embed_size)
                    OOV_embed[word] = embed
                    
            doc_embed[i] = embed

    return doc_embed

stopset = stopwords.words('english') + list(string.punctuation)

In [36]:
def calc_doc_length(doc, model, embed_size, OOV_embed):
    
    words = doc.split(" ")
    doc_len = 0
    for i in range(len(words)):
        word = words[i]
        if word not in stopset:
            try: 
                embed = model.get_vector(word)
            except:
                try:
                    embed = OOV_embed[word]
                except:
                    embed = np.random.rand(embed_size)
                    OOV_embed[word] = embed
                    
            doc_len += 1

    return doc_len

In [37]:
X_train_len = np.asarray([calc_doc_length(d, model_em, 100, OOV_embed) for d in X_train])
X_test_len = np.asarray([calc_doc_length(d, model_em, 100, OOV_embed) for d in X_test])

In [38]:
X_train_embeded = np.array([calc_doc_embedding(d, model_em, 100, OOV_embed) for d in X_train])
X_test_embeded = np.array([calc_doc_embedding(d, model_em, 100, OOV_embed) for d in X_test])

In [39]:
class CustomDataset(Dataset):
    def __init__(self, X, Y, x_length):
        self.X = X
        self.y = Y
        self.x_length = x_length
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return  torch.from_numpy(self.X[idx].astype(np.float64)), torch.from_numpy(self.y[idx].astype(np.int32)),self.x_length[idx]

In [40]:
def to_one_hot(labels, dimension=7):
  results = np.zeros(dimension)
  results[labels-1] = 1
  return results.astype(int)

y_train_embeded = y_train.apply(to_one_hot).values
y_test_embeded = y_test.apply(to_one_hot).values

In [41]:
training_set = CustomDataset(X_train_embeded, y_train_embeded, X_train_len)
testing_set = CustomDataset(X_test_embeded, y_test_embeded, X_test_len)

In [42]:
batch_size = 256
training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_set, batch_size=batch_size)

In [43]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 7)
        
    def forward(self, x, x_length):
        x_pack = pack_padded_sequence(x, x_length, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [44]:
model = LSTM_variable_input(100, 100)
model.to(device)

LSTM_variable_input(
  (lstm): LSTM(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=7, bias=True)
)

In [45]:
def BCE_loss(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [46]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=0.0001)
loss_fn = BCE_loss
loss_fu = torch.nn.CrossEntropyLoss()

In [47]:
def train(epoch):
    model.train()
    for X,y,l in training_loader:

        outputs = model(X.float().to(device),l)

        optimizer.zero_grad()

        loss = loss_fn(outputs.to(device), y.float().to(device))

        print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [48]:
EPOCHS = 100
for epoch in range(EPOCHS):
  print(epoch)
  train(epoch)

0
Epoch: 0, Loss:  0.6974338889122009
Epoch: 0, Loss:  0.6947620511054993
Epoch: 0, Loss:  0.694416880607605
Epoch: 0, Loss:  0.6918036937713623
Epoch: 0, Loss:  0.6914347410202026
Epoch: 0, Loss:  0.6902056932449341
Epoch: 0, Loss:  0.6906400918960571
Epoch: 0, Loss:  0.6885103583335876
Epoch: 0, Loss:  0.6873871088027954
Epoch: 0, Loss:  0.6857049465179443
Epoch: 0, Loss:  0.6843493580818176
Epoch: 0, Loss:  0.6851810812950134
Epoch: 0, Loss:  0.6831782460212708
Epoch: 0, Loss:  0.6823760867118835
Epoch: 0, Loss:  0.6815383434295654
Epoch: 0, Loss:  0.6797955632209778
Epoch: 0, Loss:  0.6787028908729553
Epoch: 0, Loss:  0.6776698231697083
Epoch: 0, Loss:  0.6757580637931824
Epoch: 0, Loss:  0.6753868460655212
Epoch: 0, Loss:  0.672240138053894
Epoch: 0, Loss:  0.6713458895683289
Epoch: 0, Loss:  0.6718834638595581
Epoch: 0, Loss:  0.6704393625259399
Epoch: 0, Loss:  0.6684355139732361
Epoch: 0, Loss:  0.6681812405586243
Epoch: 0, Loss:  0.6687034964561462
Epoch: 0, Loss:  0.665272474

In [49]:
def validation():
    model.eval()
    with torch.no_grad():
        for X,y,l in test_loader:
            outputs = model(X.float().to(device),l)
    return outputs, y

In [50]:
outputs, y_test = validation()

In [51]:
def output2predict(outputs, dim = 7):
  c = np.argmax(outputs)
  p = np.zeros(dim).astype(int)
  p[c] = 1
  return p

In [52]:
y_pred_em = np.array([output2predict(output.cpu()) for output in outputs])

In [53]:
accuracy = accuracy_score(y_test, y_pred_em)
microaveraged_F1_score = f1_score(y_test, y_pred_em, average='micro')
macroaveraged_F1_score = f1_score(y_test, y_pred_em, average='macro')
print("Architecture 2 accuracy:", accuracy)
print("Architecture 2 microaveraged F1 score:", microaveraged_F1_score)
print("Architecture 2 macroaveraged F1 score:", macroaveraged_F1_score)

Architecture 2 accuracy: 0.7142857142857143
Architecture 2 microaveraged F1 score: 0.7142857142857143
Architecture 2 macroaveraged F1 score: 0.5532743004633339
